# Crear JSONs

In [6]:
import pandas as pd
import os

# 1. Configuración de salida
os.makedirs('../data/processed', exist_ok=True)

def procesar_ano(df, ano):
    # Diccionario de mapeo basado en tus listas
    # Nota: He añadido 'ANYO' porque así viene en tus archivos
    mapeo = {
            'ANYO': 'ANIO',
            'COD_PROVINCIA': 'PROVINCIA',
            'TIPO_VIA': 'TIPO_VIA_NOMBRE',
            'TIPO_ACCIDENTE': 'TIPO_ACCIDENTE_NOMBRE',
            'CONDICION_METEO': 'CONDICION_METEO', 
            'CARRETERA': 'CARRETERA'
        }
    df = df.rename(columns=mapeo)
    
    # 2. Rellenar nulos para las nuevas columnas de texto (Evita errores en el modelo)
    for col in ['CONDICION_METEO', 'CARRETERA', 'DIA_SEMANA', 'CONDICION_ILUMINACION']:
        if col in df.columns:
            df[col] = df[col].fillna("DESCONOCIDO")
    # Columnas de fallecidos que me pasaste (exactas)
    fallecidos_cols = [
        'TOT_PEAT_MU24H', 'TOT_BICI_MU24H', 'TOT_CICLO_MU24H', 'TOT_MOTO_MU24H',
        'TOT_TUR_MU24H', 'TOT_FURG_MU24H', 'TOT_CAM_MENOS3500_MU24H', 
        'TOT_CAM_MAS3500_MU24H', 'TOTAL_MU24H'
    ]

    # Asegurar que existan y no tengan nulos
    for col in fallecidos_cols:
        if col not in df.columns:
            df[col] = 0
        df[col] = df[col].fillna(0)

    # Lógica de unificación para las gráficas
    df['MUERTOS_MOTO'] = df['TOT_CICLO_MU24H'] + df['TOT_MOTO_MU24H']
    df['MUERTOS_COCHE'] = df['TOT_TUR_MU24H']
    df['MUERTOS_BICI'] = df['TOT_BICI_MU24H']
    
    # Variable objetivo para el modelo y gráficas
    df['ES_MORTAL'] = (df['TOTAL_MU24H'] > 0).astype(int)
    df['ANIO'] = ano # Forzamos el año del archivo

    # Selección final de columnas para los JSON
    cols_web = [
        'ANIO', 'HORA', 'MES', 'DIA_SEMANA', 'PROVINCIA', 
        'TIPO_VIA_NOMBRE', 'TIPO_ACCIDENTE_NOMBRE', 
        'CONDICION_METEO', 'CARRETERA', 'CONDICION_ILUMINACION',
        'MUERTOS_MOTO', 'MUERTOS_COCHE', 'MUERTOS_BICI', 
        'ES_MORTAL', 'TOTAL_MU24H' # Añadimos TOTAL_MU24H para el JSON de carreteras
    ]
    
    # Solo devolvemos las que existan para evitar KeyErrors
    existentes = [c for c in cols_web if c in df.columns]
    return df[existentes]

# --- CARGA Y UNIFICACIÓN ---

archivos = {
    2020: '../data/raw/acc_2020.xlsx', 
    2021: '../data/raw/acc_2021.xlsx', 
    2022: '../data/raw/acc_2022.xlsx', 
    2023: '../data/raw/acc_2023.xlsx',
    2024: '../data/raw/acc_2024.xlsx'
}

datasets = []
for ano, path in archivos.items():
    print(f"⌛ Procesando {ano}...")
    temp_df = pd.read_excel(path, engine='openpyxl')
    datasets.append(procesar_ano(temp_df, ano))

df_historico = pd.concat(datasets, ignore_index=True)

# # --- EXPORTACIÓN JSON ---

# # 1. Evolución (Barras)
# df_historico.groupby('ANIO').agg({
#     'MUERTOS_MOTO': 'sum', 'MUERTOS_COCHE': 'sum', 
#     'MUERTOS_BICI': 'sum', 'ES_MORTAL': 'count'
# }).reset_index().to_json('../data/processed/stats_evolucion.json', orient='records')

# # 2. Hora (Línea)
# df_historico.groupby('HORA')['ES_MORTAL'].mean().reset_index().to_json('../data/processed/prob_mortalidad_hora.json', orient='records')

# print("✅ Archivos JSON generados correctamente.")

⌛ Procesando 2020...
⌛ Procesando 2021...
⌛ Procesando 2022...
⌛ Procesando 2023...
⌛ Procesando 2024...


In [5]:
# 3. Riesgo por Meteorología
df_historico.groupby('CONDICION_METEO').agg(
    Total_Accidentes=('ES_MORTAL', 'count'),
    Probabilidad_Mortal=('ES_MORTAL', 'mean')
).reset_index().to_json('../data/processed/stats_riesgo_meteo.json', orient='records')

# 4. Top 20 Carreteras (Puntos Negros)
df_historico.groupby('CARRETERA').agg(
    Total_Accidentes=('ES_MORTAL', 'count'),
    Total_Muertos=('TOTAL_MU24H', 'sum'),
    Riesgo_Tramo=('ES_MORTAL', 'mean')
).reset_index().sort_values(by='Total_Accidentes', ascending=False).head(20).to_json('../data/processed/stats_riesgo_carreteras.json', orient='records')

# 5. Tipología de Accidente de Moto
df_historico.groupby('TIPO_ACCIDENTE_NOMBRE').agg(
    Frecuencia=('ES_MORTAL', 'count'),
    Mortalidad_Media=('ES_MORTAL', 'mean')
).reset_index().to_json('../data/processed/stats_motos_tipologia.json', orient='records')

print("🚀 Todos los JSONs estratégicos han sido generados.")

NameError: name 'df_historico' is not defined

In [24]:
# 1. Filtramos accidentes donde hubo al menos un motorista fallecido o implicado
# (Usamos MUERTOS_MOTO > 0 para ver letalidad específica)
df_motos = df_historico[df_historico['MUERTOS_MOTO'] > 0].copy()

# 2. Agrupamos por tipo de accidente
stats_motos = df_historico.groupby('TIPO_ACCIDENTE_NOMBRE').agg(
    Frecuencia=('ES_MORTAL', 'count'),      # Cuántos accidentes de este tipo hay
    Tasa_Mortalidad=('ES_MORTAL', 'mean')   # Probabilidad de que sea mortal
).reset_index()

# Filtramos tipos con muy pocos casos para que la gráfica no sea un caos
stats_motos = stats_motos[stats_motos['Frecuencia'] > 20]

stats_motos.to_json('../data/processed/stats_motos_tipologia.json', orient='records')

In [25]:
stats_provincias = df_historico.groupby('PROVINCIA').agg(
    Total_Accidentes=('ES_MORTAL', 'count'),
    Muertos_Moto=('MUERTOS_MOTO', 'sum'),
    Riesgo_Medio=('ES_MORTAL', 'mean')
).reset_index()

stats_provincias.to_json('../data/processed/stats_provincias.json', orient='records')

In [26]:
stats_meses = df_historico.groupby('MES').agg(
    Muertos_Moto=('MUERTOS_MOTO', 'sum'),
    Muertos_Coche=('MUERTOS_COCHE', 'sum')
).reset_index()

stats_meses.to_json('../data/processed/stats_meses.json', orient='records')

In [10]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

# --- 1. FEATURE ENGINEERING (Mejora de datos) ---
def crear_features(df):
    df = df.copy()
    
    # A. Variable Fin de Semana mejorada
    # Si DIA_SEMANA son números (1=Lunes...7=Domingo), el fin de semana es 6 y 7
    # Si son strings, buscamos las palabras.
    if pd.api.types.is_numeric_dtype(df['DIA_SEMANA']):
        df['ES_FIN_SEMANA'] = df['DIA_SEMANA'].isin([6, 7]).astype(int)
    else:
        dias_festivos = ['SÁBADO', 'DOMINGO', 'SABADO']
        df['ES_FIN_SEMANA'] = df['DIA_SEMANA'].str.upper().isin(dias_festivos).astype(int)
    
    # B. Variable Nocturna
    df['ES_NOCTURNO'] = ((df['HORA'] >= 22) | (df['HORA'] <= 6)).astype(int)
    
    # C. Variable Riesgo Visibilidad
    # Aseguramos que tratamos con strings para evitar errores similares
    meteo_mala = ['LLUVIA', 'NIEVE', 'GRANIZO']
    cond_meteo = df['CONDICION_METEO'].astype(str).str.upper()
    cond_ilum = df['CONDICION_ILUMINACION'].astype(str).str.upper()
    
    df['RIESGO_VISIBILIDAD'] = ((cond_meteo.isin(meteo_mala)) & 
                                (cond_ilum != 'PLENO DÍA')).astype(int)
    return df

df_enriquecido = crear_features(df_historico)

# --- 2. DEFINICIÓN DE VARIABLES ---
features = [
    'HORA', 'MES', 'DIA_SEMANA', 'PROVINCIA', 'TIPO_VIA_NOMBRE', 
    'TIPO_ACCIDENTE_NOMBRE', 'CONDICION_METEO', 'CARRETERA', 
    'CONDICION_ILUMINACION', 'ES_FIN_SEMANA', 'ES_NOCTURNO', 'RIESGO_VISIBILIDAD'
]

X = df_enriquecido[features]
y = df_enriquecido['ES_MORTAL']

categorical_features = [
    'DIA_SEMANA', 'PROVINCIA', 'TIPO_VIA_NOMBRE', 
    'TIPO_ACCIDENTE_NOMBRE', 'CONDICION_METEO', 
    'CARRETERA', 'CONDICION_ILUMINACION'
]

# --- 3. PIPELINE CON XGBOOST (Más potente que RandomForest) ---
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
    ],
    remainder='passthrough'
)

modelo_vial_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        scale_pos_weight=(len(y) - sum(y)) / sum(y), # Balanceo automático
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    ))
])

# --- 4. ENTRENAMIENTO ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("🧠 Entrenando modelo avanzado XGBoost...")
modelo_vial_pipeline.fit(X_train, y_train)

# --- 5. GUARDAR Y MOSTRAR STATS ---
joblib.dump(modelo_vial_pipeline, '../data/processed/modelo_vial_v2.pkl')

# Métricas detalladas
y_pred = modelo_vial_pipeline.predict(X_test)
accuracy = modelo_vial_pipeline.score(X_test, y_test)

print("\n" + "="*40)
print(f"📊 ESTADÍSTICAS DEL MODELO V2")
print("="*40)
print(f"✅ Precisión General: {accuracy:.4f}")
print("\n📝 Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Importancia de las variables (¿Qué influye más?)
importances = modelo_vial_pipeline.named_steps['classifier'].feature_importances_
feature_names = categorical_features + ['HORA', 'MES', 'ES_FIN_SEMANA', 'ES_NOCTURNO', 'RIESGO_VISIBILIDAD']
sorted_idx = np.argsort(importances)[::-1]

print("\n🔍 TOP 5 FACTORES DETERMINANTES:")
for i in range(5):
    print(f"{i+1}. {feature_names[sorted_idx[i]]}: {importances[sorted_idx[i]]:.4f}")
print("="*40)

🧠 Entrenando modelo avanzado XGBoost...


c:\Users\ELOY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [22:21:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 ESTADÍSTICAS DEL MODELO V2
✅ Precisión General: 0.8626

📝 Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      0.87      0.93     91502
           1       0.05      0.49      0.09      1306

    accuracy                           0.86     92808
   macro avg       0.52      0.68      0.51     92808
weighted avg       0.98      0.86      0.91     92808


🔍 TOP 5 FACTORES DETERMINANTES:
1. TIPO_VIA_NOMBRE: 0.3324
2. TIPO_ACCIDENTE_NOMBRE: 0.1908
3. ES_NOCTURNO: 0.0808
4. CONDICION_ILUMINACION: 0.0744
5. PROVINCIA: 0.0668


## 📊 Informe de Rendimiento del Modelo Predictivo (v2 - XGBoost)

Tras procesar el histórico de accidentes (2020-2024) y aplicar técnicas de **Feature Engineering**, el modelo ha sido optimizado utilizando un algoritmo de **Extreme Gradient Boosting (XGBoost)**.

### 📈 Métricas de Clasificación
* **Precisión General (Accuracy):** `86.26%`
* **Recall (Clase Mortal):** `0.49` 
    > *Nota: El modelo prioriza la detección de riesgos (Sensibilidad) sobre la precisión pura, logrando identificar casi el 50% de los eventos mortales en un dataset altamente desbalanceado.*

### 🔍 Factores Determinantes (Feature Importance)
El modelo ha identificado los siguientes factores como los más influyentes en la probabilidad de mortalidad de un accidente:

| Ranking | Variable | Peso (Importancia) | Descripción |
| :--- | :--- | :--- | :--- |
| 1º | **TIPO_VIA_NOMBRE** | **33.24%** | El diseño y velocidad de la vía es el factor crítico. |
| 2º | **TIPO_ACCIDENTE_NOMBRE** | **19.08%** | La dinámica del impacto (choque, salida de vía, etc.). |
| 3º | **ES_NOCTURNO** | **8.08%** | Variable creada (Feature Eng.) que confirma el riesgo entre 22:00 y 06:00. |
| 4º | **CONDICION_ILUM** | **7.44%** | La visibilidad artificial o natural en el momento del siniestro. |
| 5º | **PROVINCIA** | **6.68%** | El factor geográfico y de gestión de infraestructuras locales. |



### 🛠️ Mejoras Implementadas
1.  **Ingeniería de Variables:** Creación de indicadores sintéticos como `ES_NOCTURNO` (franja horaria de riesgo) y `RIESGO_VISIBILIDAD` (cruce de meteorología e iluminación).
2.  **Tratamiento de Desbalanceo:** Implementación de `scale_pos_weight` para ajustar el aprendizaje ante la baja frecuencia de accidentes mortales.
3.  **Normalización:** Pipeline automatizado para la codificación de variables categóricas y gestión de valores desconocidos.

---